In [ ]:
# ============================================================
# ESPN Fantasy Draft Value Project (Re-ordered, end-to-end)
# - Block-by-block, in a logical pipeline
# - Separates: (A) hygiene + load, (B) league-year filtering,
#              (C) enrichment, (D) optimal lineup valid points,
#              (E) autodraft baseline + scoring, (F) plots
# - Adds: scoring-rule filter using anchor players
# ============================================================

from __future__ import annotations

from pathlib import Path
import re
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# ============================================================
# BLOCK 0 — CONFIG
# ============================================================

RAW_BASE = Path("data/raw/espn")                  # expects subfolders like 2021/, 2022/, ...
OUT_DIR  = Path("data/preprocessed")
OUT_DIR.mkdir(parents=True, exist_ok=True)

YEARS = range(2021, 2025)                         # your current dataset range
RAW_FILES = ["lineup_data.csv", "draft_data.csv", "transaction_data.csv"]

EXPECTED_STARTERS = {"QB":1,"RB":2,"WR":2,"TE":1,"FLEX":1,"K":1,"D/ST":1,"OP":0,"STARTERS":9}
EXPECTED_DRAFT_LENGTH = 160

# "Anchor players" for scoring-rule detection:
# pick players that appear most seasons + are usually drafted/rostered.
# You can tune these later.
ANCHORS_BY_POS = {
    "QB":  ["Josh Allen", "Patrick Mahomes"],
    "RB":  ["Christian McCaffrey", "Derrick Henry"],
    "WR":  ["Justin Jefferson", "Tyreek Hill"],
    "TE":  ["Travis Kelce", "Mark Andrews"],
    "K":   ["Justin Tucker"],
    "D/ST":["Patriots D/ST", "49ers D/ST", "Cowboys D/ST"],  # depends on naming in your data
}
# If anchor names don’t match your dataset strings, add synonyms below.
ANCHOR_SYNONYMS = {
    # "New England Patriots": "Patriots D/ST",
    # "San Francisco 49ers": "49ers D/ST",
}

In [ ]:
# ============================================================
# BLOCK 1 — RAW FILE HYGIENE: DROP DUPLICATE ROWS IN EACH YEAR CSV
# ============================================================

def drop_raw_duplicates_in_place(base_dir: Path, years, files) -> None:
    for year in years:
        print(f"\n=== YEAR {year} ===")
        year_dir = base_dir / str(year)

        for fname in files:
            path = year_dir / fname
            if not path.exists():
                print(f"  {fname}: missing")
                continue

            df = pd.read_csv(path)
            before = len(df)
            df2 = df.drop_duplicates()
            after = len(df2)
            dropped = before - after

            if dropped == 0:
                print(f"  {fname}: no duplicates ({before:,} rows)")
            else:
                df2.to_csv(path, index=False)
                print(f"  {fname}: dropped {dropped:,} duplicate rows ({before:,} → {after:,})")


# Run once (safe to re-run)
drop_raw_duplicates_in_place(RAW_BASE, YEARS, RAW_FILES)

In [ ]:
# ============================================================
# BLOCK 2 — NORMALIZATION HELPERS
# ============================================================

CORE_POS = {"QB", "RB", "WR", "TE", "K", "D/ST", "DST", "DEF"}

def normalize_player_name(name: str) -> str:
    if pd.isna(name):
        return name
    s = str(name).strip()
    s = re.sub(r"\s+", " ", s)
    s = s.replace(" Jr.", "").replace(" Sr.", "")
    # apply anchor synonyms (optional)
    s = ANCHOR_SYNONYMS.get(s, s)
    return s

def normalize_slot(slot: str) -> str:
    s = str(slot).strip().upper()
    if s in {"DST", "DEF", "D/ST"}:
        return "D/ST"
    if s in {"BE", "BENCH"}:
        return "BE"
    if s in {"IR"}:
        return "IR"
    if s in {"RB/WR/TE", "FLEX"}:
        return "FLEX"
    if s in {"OP", "SUPERFLEX", "QB/RB/WR/TE"}:
        return "OP"
    return s

In [ ]:
# ============================================================
# BLOCK 3 — LOADERS (multi-season draft + lineups)
# ============================================================

def load_draft(path: Path, year: int) -> pd.DataFrame:
    df = pd.read_csv(path)
    required = ["League_ID","Player","Team","Round","Pick","Overall","Is_Autodrafted","Auto_Draft_Type_ID"]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"{path} draft_data is missing columns: {missing}")

    df = df.copy()
    df["Year"] = int(year)
    df["Player_norm"] = df["Player"].map(normalize_player_name)
    df["League_ID"] = df["League_ID"].astype(int)
    df["Year"] = df["Year"].astype(int)
    df["Overall"] = pd.to_numeric(df["Overall"], errors="coerce")
    df["Is_Autodrafted"] = pd.to_numeric(df["Is_Autodrafted"], errors="coerce").fillna(0).astype(int)
    return df

def load_lineups(path: Path, year: int) -> pd.DataFrame:
    df = pd.read_csv(path)
    required = ["League_ID","Week","Team","Player","Slot","Points"]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"{path} lineup_data is missing columns: {missing}")

    df = df.copy()
    df["Year"] = int(year)
    df["Player_norm"] = df["Player"].map(normalize_player_name)
    df["League_ID"] = df["League_ID"].astype(int)
    df["Year"] = df["Year"].astype(int)
    df["Week"] = pd.to_numeric(df["Week"], errors="coerce")
    df["Points"] = pd.to_numeric(df["Points"], errors="coerce")
    df["Slot"] = df["Slot"].astype(str).str.strip()
    if "Is_Starter" in df.columns:
        df["Is_Starter"] = pd.to_numeric(df["Is_Starter"], errors="coerce")
    return df

def load_multi_season(raw_base: Path) -> tuple[pd.DataFrame, pd.DataFrame]:
    draft_parts = []
    lineup_parts = []

    for year_dir in sorted([p for p in raw_base.iterdir() if p.is_dir() and p.name.isdigit() and len(p.name) == 4]):
        year = int(year_dir.name)

        dpath = year_dir / "draft_data.csv"
        lpath = year_dir / "lineup_data.csv"

        if dpath.exists():
            draft_parts.append(load_draft(dpath, year))
        else:
            print(f"Skipping {year}: missing {dpath}")

        if lpath.exists():
            lineup_parts.append(load_lineups(lpath, year))
        else:
            print(f"Skipping {year}: missing {lpath}")

    if not draft_parts:
        raise FileNotFoundError(f"No draft_data.csv found under {raw_base}/<YEAR>/draft_data.csv")
    if not lineup_parts:
        raise FileNotFoundError(f"No lineup_data.csv found under {raw_base}/<YEAR>/lineup_data.csv")

    draft_all = pd.concat(draft_parts, ignore_index=True)
    lineups_all = pd.concat(lineup_parts, ignore_index=True)
    return draft_all, lineups_all


draft_raw, lineups_raw = load_multi_season(RAW_BASE)

In [ ]:
# ============================================================
# BLOCK 4 — BASIC DRAFT METADATA
# ============================================================

def add_draft_length(draft_df: pd.DataFrame) -> pd.DataFrame:
    out = draft_df.copy()
    out["Draft_Length"] = (
        out.groupby(["League_ID", "Year"], dropna=False)["Overall"]
          .transform("max")
    )
    out["Draft_Length"] = pd.to_numeric(out["Draft_Length"], errors="coerce").fillna(-1).astype(int)
    return out

draft_raw = add_draft_length(draft_raw)

In [ ]:
# ============================================================
# BLOCK 5 — FILTER #1: STANDARD STARTER SIGNATURE (league-year)
# ============================================================

def infer_league_starter_signature(lineups: pd.DataFrame) -> pd.DataFrame:
    df = lineups.copy()
    required = {"League_ID","Year","Team","Week","Slot","Is_Starter"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"lineups missing columns for starter signature: {sorted(missing)}")

    df["Slot_norm"] = df["Slot"].map(normalize_slot)
    starters = df[df["Is_Starter"].fillna(0).astype(int) == 1].copy()

    tw = (
        starters.groupby(["League_ID","Year","Team","Week","Slot_norm"])
        .size()
        .reset_index(name="n")
    )

    pivot = (
        tw.pivot_table(
            index=["League_ID","Year","Team","Week"],
            columns="Slot_norm",
            values="n",
            aggfunc="sum",
            fill_value=0,
        )
        .reset_index()
    )

    for col in ["QB","RB","WR","TE","K","D/ST","FLEX","OP"]:
        if col not in pivot.columns:
            pivot[col] = 0

    pivot["Starters_Total"] = pivot[["QB","RB","WR","TE","K","D/ST","FLEX","OP"]].sum(axis=1)

    def mode_int(s: pd.Series) -> int:
        vc = s.value_counts()
        return int(vc.index[0]) if len(vc) else 0

    league_sig = (
        pivot.groupby(["League_ID","Year"], dropna=False)
        .agg(
            QB=("QB", mode_int),
            RB=("RB", mode_int),
            WR=("WR", mode_int),
            TE=("TE", mode_int),
            K=("K", mode_int),
            DST=("D/ST", mode_int),
            FLEX=("FLEX", mode_int),
            OP=("OP", mode_int),
            Starters_Total=("Starters_Total", mode_int),
        )
        .reset_index()
    )

    league_sig["Signature"] = (
        "QB=" + league_sig["QB"].astype(str) +
        ",RB=" + league_sig["RB"].astype(str) +
        ",WR=" + league_sig["WR"].astype(str) +
        ",TE=" + league_sig["TE"].astype(str) +
        ",FLEX=" + league_sig["FLEX"].astype(str) +
        ",K=" + league_sig["K"].astype(str) +
        ",DST=" + league_sig["DST"].astype(str) +
        ",OP=" + league_sig["OP"].astype(str) +
        ",START=" + league_sig["Starters_Total"].astype(str)
    )
    return league_sig

def filter_standard_leagues(
    draft: pd.DataFrame,
    lineups: pd.DataFrame,
    *,
    expected: dict[str, int] | None = None,
    verbose: bool = True
) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    if expected is None:
        expected = {"QB":1, "RB":2, "WR":2, "TE":1, "FLEX":1, "K":1, "D/ST":1, "OP":0, "STARTERS":9}

    league_sig = infer_league_starter_signature(lineups)

    keep = (
        (league_sig["QB"] == expected["QB"]) &
        (league_sig["RB"] == expected["RB"]) &
        (league_sig["WR"] == expected["WR"]) &
        (league_sig["TE"] == expected["TE"]) &
        (league_sig["FLEX"] == expected["FLEX"]) &
        (league_sig["K"] == expected["K"]) &
        (league_sig["DST"] == expected["D/ST"]) &
        (league_sig["OP"] == expected["OP"]) &
        (league_sig["Starters_Total"] == expected["STARTERS"])
    )
    league_sig["Keep_Standard"] = keep

    kept = league_sig[league_sig["Keep_Standard"]][["League_ID","Year"]]
    draft_filt  = draft.merge(kept, on=["League_ID","Year"], how="inner")
    lineups_filt = lineups.merge(kept, on=["League_ID","Year"], how="inner")

    if verbose:
        total = len(league_sig)
        k = int(league_sig["Keep_Standard"].sum())
        print(f"[filter_standard_leagues] keeping {k}/{total} league-years ({k/total:.1%})")
        if total - k > 0:
            print("[filter_standard_leagues] top non-standard signatures:")
            print(league_sig.loc[~league_sig["Keep_Standard"], "Signature"].value_counts().head(10))

    return draft_filt, lineups_filt, league_sig


draft_std, lineups_std, league_sig = filter_standard_leagues(
    draft_raw, lineups_raw,
    expected=EXPECTED_STARTERS,
    verbose=True
)

In [ ]:
# ============================================================
# BLOCK 6 — FILTER #2: SCORING-RULE FILTER USING ANCHOR PLAYERS (league-year)
# ============================================================

def _season_points_by_league_year(lineups: pd.DataFrame) -> pd.DataFrame:
    df = lineups.copy()
    df["Points"] = pd.to_numeric(df["Points"], errors="coerce")
    return (
        df.groupby(["League_ID","Year","Player_norm"], dropna=False)["Points"]
          .sum(min_count=1)
          .reset_index()
          .rename(columns={"Points":"Season_Total_Points"})
    )

def filter_scoring_rule_outliers(
    draft: pd.DataFrame,
    lineups: pd.DataFrame,
    *,
    anchors_by_pos: dict[str, list[str]] | None = None,
    z_thresh: float = 3.5,
    min_anchors_hit: int = 4,
    verbose: bool = True
) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Goal: remove league-years with strange scoring rules.
    Method:
      - compute Season_Total_Points per (League_ID, Year, Player_norm)
      - for each position group, look at anchor player season totals across league-years
      - compute robust z-score via median/MAD per (Year, anchor)
      - mark league-year as outlier if too many anchors are extreme
    Notes:
      - This is intentionally simple + robust.
      - Works even if you don't have explicit scoring settings.
    """
    if anchors_by_pos is None:
        anchors_by_pos = ANCHORS_BY_POS

    # build season totals
    pts = _season_points_by_league_year(lineups)

    # build anchor table
    anchor_rows = []
    for pos, names in anchors_by_pos.items():
        for n in names:
            anchor_rows.append((pos, normalize_player_name(n)))
    anchors = pd.DataFrame(anchor_rows, columns=["AnchorPos","Player_norm"]).drop_duplicates()

    # join to season totals
    a = pts.merge(anchors, on="Player_norm", how="inner")
    if a.empty:
        meta = pd.DataFrame(columns=["League_ID","Year","Anchors_Hit","Anchors_Outlier","Drop"])
        return draft, lineups, meta

    # robust z per (Year, Player_norm)
    def robust_z(s: pd.Series) -> pd.Series:
        x = s.astype(float)
        med = np.nanmedian(x)
        mad = np.nanmedian(np.abs(x - med))
        if mad == 0 or np.isnan(mad):
            return (x - med) * 0.0
        return 0.6745 * (x - med) / mad  # ~ std units for normal

    a["rz"] = a.groupby(["Year","Player_norm"], dropna=False)["Season_Total_Points"].transform(robust_z)
    a["Is_Outlier"] = a["rz"].abs() >= z_thresh

    # league-year summary
    meta = (
        a.groupby(["League_ID","Year"], dropna=False)
         .agg(
            Anchors_Hit=("Player_norm","nunique"),
            AnchorRows=("Player_norm","size"),
            Anchors_Outlier=("Is_Outlier","sum"),
            MaxAbsRZ=("rz", lambda s: float(np.nanmax(np.abs(s.to_numpy()))) if len(s) else np.nan),
         )
         .reset_index()
    )

    # drop if: too many outliers OR not enough anchors observed
    meta["Drop"] = (meta["Anchors_Hit"] < min_anchors_hit) | (meta["Anchors_Outlier"] >= 1)

    kept = meta[~meta["Drop"]][["League_ID","Year"]]
    draft_filt = draft.merge(kept, on=["League_ID","Year"], how="inner")
    lineups_filt = lineups.merge(kept, on=["League_ID","Year"], how="inner")

    if verbose:
        total = meta.shape[0]
        kept_n = kept.shape[0]
        print(f"[filter_scoring_rule_outliers] keeping {kept_n}/{total} league-years ({kept_n/total:.1%})")
        if total - kept_n > 0:
            print("[filter_scoring_rule_outliers] examples of dropped league-years:")
            print(meta[meta["Drop"]].sort_values(["Anchors_Hit","Anchors_Outlier","MaxAbsRZ"], ascending=[True,False,False]).head(10))

    return draft_filt, lineups_filt, meta


draft_filt, lineups_filt, scoring_meta = filter_scoring_rule_outliers(
    draft_std, lineups_std,
    z_thresh=3.5,
    min_anchors_hit=4,
    verbose=True
)

In [ ]:
# ============================================================
# BLOCK 7 — OPTIONAL FILTER #3: DRAFT LENGTH (league-year)
# ============================================================

def filter_draft_length(draft: pd.DataFrame, lineups: pd.DataFrame, *, length: int, verbose: bool = True):
    d = draft.copy()
    if "Draft_Length" not in d.columns:
        d = add_draft_length(d)

    kept = (
        d.groupby(["League_ID","Year"], dropna=False)["Draft_Length"]
         .max()
         .reset_index()
    )
    kept = kept[kept["Draft_Length"] == int(length)][["League_ID","Year"]]

    d2 = d.merge(kept, on=["League_ID","Year"], how="inner")
    l2 = lineups.merge(kept, on=["League_ID","Year"], how="inner")

    if verbose:
        total = d.groupby(["League_ID","Year"]).ngroups
        kept_n = kept.shape[0]
        print(f"[filter_draft_length] keeping {kept_n}/{total} league-years ({kept_n/total:.1%}) with Draft_Length={length}")

    return d2, l2


draft_filt, lineups_filt = filter_draft_length(
    draft_filt, lineups_filt,
    length=EXPECTED_DRAFT_LENGTH,
    verbose=True
)

In [ ]:
# ============================================================
# BLOCK 8 — ENRICHMENT: POSITION + SEASON TOTAL POINTS (simple)
# ============================================================

def infer_position_from_slots(lineups: pd.DataFrame) -> pd.DataFrame:
    x = lineups.copy()
    x["Slot_norm"] = x["Slot"].map(normalize_slot)
    x["Is_core_pos"] = x["Slot_norm"].isin({"QB","RB","WR","TE","K","D/ST"})

    counts = (
        x.groupby(["League_ID","Year","Player_norm","Slot_norm","Is_core_pos"], dropna=False)
         .size().reset_index(name="n")
         .sort_values(by=["League_ID","Year","Player_norm","Is_core_pos","n"],
                      ascending=[True,True,True,False,False])
    )

    top = (
        counts.drop_duplicates(subset=["League_ID","Year","Player_norm"])
              .rename(columns={"Slot_norm":"Position"})[["League_ID","Year","Player_norm","Position"]]
    )
    return top

def season_points_from_lineups(lineups: pd.DataFrame) -> pd.DataFrame:
    df = lineups.copy()
    df["Points"] = pd.to_numeric(df["Points"], errors="coerce")
    return (
        df.groupby(["League_ID","Year","Player_norm"], dropna=False)["Points"]
          .sum(min_count=1)
          .reset_index()
          .rename(columns={"Points":"Season_Total_Points"})
    )

def build_draft_enriched(draft: pd.DataFrame, lineups: pd.DataFrame) -> pd.DataFrame:
    d = draft.copy()
    d["Player_norm"] = d["Player_norm"].map(normalize_player_name)

    pos = infer_position_from_slots(lineups)
    pts = season_points_from_lineups(lineups)

    out = d.merge(pos, on=["League_ID","Year","Player_norm"], how="left")
    out = out.merge(pts, on=["League_ID","Year","Player_norm"], how="left")

    out["Season_Total_Points"] = pd.to_numeric(out["Season_Total_Points"], errors="coerce").fillna(0.0)

    # keep a clean set of columns
    cols = [
        "League_ID","Year","Team","Player",
        "Round","Pick","Overall",
        "Draft_Length",
        "Position","Season_Total_Points",
        "Is_Autodrafted","Auto_Draft_Type_ID",
        "Player_norm",
    ]
    for c in cols:
        if c not in out.columns:
            out[c] = np.nan
    return out[cols].copy()


draft_enriched = build_draft_enriched(draft_filt, lineups_filt)
draft_enriched_path = OUT_DIR / "draft_enriched_filtered.csv"
draft_enriched.to_csv(draft_enriched_path, index=False)
print(f"[saved] {draft_enriched_path} ({len(draft_enriched):,} rows)")



In [ ]:
# ============================================================
# BLOCK 9 — OPTIMAL STARTABLE POINTS (“VALID POINTS”) PIPELINE
#   (your slow-but-correct approach, cleaned up)
# ============================================================

def _normalize_slot_defense(slot: str) -> str:
    slot = str(slot).strip()
    return {"DST": "D/ST", "DEF": "D/ST"}.get(slot, slot)

def infer_player_position_by_core_starts(lineups: pd.DataFrame) -> pd.DataFrame:
    CORE = {"QB", "RB", "WR", "TE", "K", "D/ST"}
    df = lineups.copy()
    df["Slot"] = df["Slot"].map(_normalize_slot_defense)

    if "Year" not in df.columns:
        raise ValueError("lineups must include a 'Year' column for multi-season processing.")

    core_rows = df[df["Slot"].isin(CORE)].copy()
    if core_rows.empty:
        raise ValueError("No core slots found in lineup_data Slot column (QB/RB/WR/TE/K/DST).")

    pos = (
        core_rows.groupby(["League_ID", "Year", "Team", "Player_norm"])["Slot"]
        .agg(lambda s: s.value_counts().index[0])
        .reset_index()
        .rename(columns={"Slot": "Position"})
    )
    return pos

def build_player_week_points(lineups: pd.DataFrame) -> pd.DataFrame:
    df = lineups.copy()
    df["Slot"] = df["Slot"].map(_normalize_slot_defense)
    df["Points"] = pd.to_numeric(df["Points"], errors="coerce").fillna(0.0)

    if "Year" not in df.columns:
        raise ValueError("lineups must include a 'Year' column for multi-season processing.")

    pw = (
        df.groupby(["League_ID", "Year", "Team", "Week", "Player_norm"], dropna=False)["Points"]
        .max()
        .reset_index()
        .rename(columns={"Points": "WeekPoints"})
    )
    return pw

def choose_optimal_lineup_for_group(
    g: pd.DataFrame,
    slot_counts: dict[str,int],
    flex_eligible: set[str]
) -> pd.DataFrame:
    g = g.copy()
    g["SelectedOptimal"] = False
    used = set()

    def select_top(pos, n):
        nonlocal used
        if n <= 0:
            return []
        cand = g[(g["Position"] == pos) & (~g["Player_norm"].isin(used))].sort_values("WeekPoints", ascending=False)
        chosen = cand.head(n)["Player_norm"].tolist()
        used.update(chosen)
        return chosen

    def select_flex(n):
        nonlocal used
        if n <= 0:
            return []
        cand = g[(g["Position"].isin(flex_eligible)) & (~g["Player_norm"].isin(used))].sort_values("WeekPoints", ascending=False)
        chosen = cand.head(n)["Player_norm"].tolist()
        used.update(chosen)
        return chosen

    selected = []
    for pos in ["QB", "RB", "WR", "TE", "K", "D/ST"]:
        selected += select_top(pos, slot_counts.get(pos, 0))
    selected += select_flex(slot_counts.get("FLEX", 0))

    g.loc[g["Player_norm"].isin(selected), "SelectedOptimal"] = True
    return g

def compute_optimal_startable_points(
    lineups: pd.DataFrame,
    *,
    slot_counts: dict[str,int] | None = None,
    flex_eligible: set[str] | None = None,
    status_every: int = 250
) -> pd.DataFrame:
    if slot_counts is None:
        slot_counts = {"QB":1, "RB":2, "WR":2, "TE":1, "FLEX":1, "K":1, "D/ST":1}
    if flex_eligible is None:
        flex_eligible = {"RB","WR","TE"}

    t_all = time.time()
    print("[compute_optimal_startable_points] start", flush=True)

    df = lineups.copy()
    df["Player_norm"] = df["Player_norm"] if "Player_norm" in df.columns else df["Player"].astype(str).str.strip()
    df["Slot"] = df["Slot"].map(_normalize_slot_defense)

    if "Year" not in df.columns:
        raise ValueError("lineups must include a 'Year' column for multi-season processing.")

    print(f"[compute_optimal_startable_points] rows: {len(df):,}", flush=True)

    pos = infer_player_position_by_core_starts(df)
    pw = build_player_week_points(df)

    pw = pw.merge(pos, on=["League_ID","Year","Team","Player_norm"], how="left")
    pw["Position"] = pw["Position"].fillna("UNKNOWN")

    gb = pw.groupby(["League_ID","Year","Team","Week"], sort=False)
    n_groups = gb.ngroups
    print(f"[compute_optimal_startable_points] optimizing {n_groups:,} team-weeks...", flush=True)

    t0 = time.time()
    selected_parts = []
    for i, (_, g) in enumerate(gb, start=1):
        if i % status_every == 0 or i == 1 or i == n_groups:
            elapsed = time.time() - t0
            rate = i / elapsed if elapsed > 0 else float("inf")
            print(f"  - progress: {i:,}/{n_groups:,} groups ({rate:.1f} groups/s)", flush=True)
        selected_parts.append(choose_optimal_lineup_for_group(g, slot_counts, flex_eligible))

    selected = pd.concat(selected_parts, ignore_index=True)
    print(f"[compute_optimal_startable_points] done in {time.time()-t_all:.2f}s", flush=True)
    return selected

def add_season_total_points_valid(
    draft_enriched: pd.DataFrame,
    optimal_selected: pd.DataFrame
) -> pd.DataFrame:
    for col in ["Year", "League_ID", "Team", "Player_norm"]:
        if col not in draft_enriched.columns:
            raise ValueError(f"draft_enriched missing required column '{col}' for multi-season merge.")
    if "Year" not in optimal_selected.columns:
        raise ValueError("optimal_selected missing 'Year' column.")

    valid = (
        optimal_selected[optimal_selected["SelectedOptimal"]]
        .groupby(["League_ID","Year","Team","Player_norm"], dropna=False)["WeekPoints"]
        .sum()
        .reset_index()
        .rename(columns={"WeekPoints":"Season_Total_Points_Valid"})
    )

    out = draft_enriched.merge(valid, on=["League_ID","Year","Team","Player_norm"], how="left")
    out["Season_Total_Points_Valid"] = pd.to_numeric(out["Season_Total_Points_Valid"], errors="coerce").fillna(0.0)
    return out


optimal_selected = compute_optimal_startable_points(
    lineups_filt,
    slot_counts={"QB":1,"RB":2,"WR":2,"TE":1,"FLEX":1,"K":1,"D/ST":1},
    flex_eligible={"RB","WR","TE"},
    status_every=250
)

draft_with_valid = add_season_total_points_valid(draft_enriched, optimal_selected)
draft_with_valid_path = OUT_DIR / "draft_with_valid_points_filtered.csv"
draft_with_valid.to_csv(draft_with_valid_path, index=False)
print(f"[saved] {draft_with_valid_path} ({len(draft_with_valid):,} rows)")


In [ ]:
# ============================================================
# BLOCK 10 — FULLY AUTODRAFTED TEAMS + SANITY CHECK (1.01 variety)
# ============================================================

def identify_fully_autodrafted_teams(
    draft_df: pd.DataFrame,
    *,
    sanity_check_pick1: bool = True,
    verbose: bool = True
) -> pd.DataFrame:
    required = ["League_ID", "Year", "Team", "Is_Autodrafted"]
    missing = [c for c in required if c not in draft_df.columns]
    if missing:
        raise ValueError(f"draft_df missing columns: {missing}")

    result = (
        draft_df
        .groupby(["League_ID", "Year", "Team"], dropna=False)
        .agg(
            Total_Picks=("Is_Autodrafted", "size"),
            Autodrafted_Picks=("Is_Autodrafted", "sum")
        )
        .reset_index()
    )
    result["Is_Fully_Autodrafted"] = (result["Total_Picks"] == result["Autodrafted_Picks"])

    if sanity_check_pick1:
        for c in ["Overall", "Player_norm"]:
            if c not in draft_df.columns:
                raise ValueError(f"Sanity check requires column '{c}'")

        df_auto = draft_df.merge(
            result[["League_ID", "Year", "Team", "Is_Fully_Autodrafted"]],
            on=["League_ID", "Year", "Team"],
            how="left"
        )

        pick1 = df_auto[(df_auto["Is_Fully_Autodrafted"]) & (df_auto["Overall"] == 1)]
        if verbose:
            print("\n[Sanity Check] Fully-Autodrafted 1.01 Picks")
            if pick1.empty:
                print("  none found")
            else:
                summary = (pick1.groupby("Year")["Player_norm"]
                           .agg(Unique_1OA="nunique", Total_1OA="count")
                           .reset_index())
                print(summary.to_string(index=False))
                print(f"\nAcross all years:\n  Total 1.01 picks = {len(pick1)}\n  Unique players  = {pick1['Player_norm'].nunique()}")

    return result


fully_auto = identify_fully_autodrafted_teams(draft_with_valid, sanity_check_pick1=True, verbose=True)



In [ ]:
# ============================================================
# BLOCK 11 — TEAM TOTAL VALID POINTS DISTRIBUTIONS
# ============================================================

def team_total_valid_points(draft_with_valid: pd.DataFrame, *, fully_autodraft_only: bool | None = None) -> pd.DataFrame:
    required = ["League_ID", "Year", "Team", "Is_Autodrafted", "Season_Total_Points_Valid"]
    missing = [c for c in required if c not in draft_with_valid.columns]
    if missing:
        raise ValueError(f"draft_with_valid missing columns: {missing}")

    df = draft_with_valid.copy()

    team_auto = (
        df.groupby(["League_ID", "Year", "Team"], dropna=False)["Is_Autodrafted"]
          .agg(Total_Picks="size", Autodrafted_Picks="sum")
          .reset_index()
    )
    team_auto["Is_Fully_Autodrafted"] = team_auto["Total_Picks"].eq(team_auto["Autodrafted_Picks"])
    df = df.merge(team_auto[["League_ID", "Year", "Team", "Is_Fully_Autodrafted"]],
                  on=["League_ID", "Year", "Team"], how="left")

    if fully_autodraft_only is True:
        df = df[df["Is_Fully_Autodrafted"]].copy()
    elif fully_autodraft_only is False:
        df = df[~df["Is_Fully_Autodrafted"]].copy()

    team_totals = (
        df.groupby(["League_ID", "Year", "Team"], dropna=False)["Season_Total_Points_Valid"]
          .sum()
          .reset_index()
          .rename(columns={"Season_Total_Points_Valid": "Total_Valid_Points"})
          .sort_values("Total_Valid_Points", ascending=False)
    )
    if team_totals.empty:
        raise ValueError("No teams found after filtering.")
    return team_totals

def plot_total_valid_points_dist(team_totals: pd.DataFrame, *, bins: int = 20, title: str = "") -> None:
    x = team_totals["Total_Valid_Points"].astype(float).to_numpy()
    avg = float(np.mean(x))
    plt.figure(figsize=(10, 5))
    plt.hist(x, bins=bins, density=True, alpha=0.6)
    plt.axvline(avg, linestyle="--")
    plt.xlabel("Total Valid Points (Optimal-Start Weeks Only)")
    plt.ylabel("Density")
    plt.title(title or f"Total Valid Points Distribution (n={len(x)})")
    ymin, ymax = plt.ylim()
    plt.text(avg, ymax * 0.95, f"avg = {avg:.1f}", va="top")
    plt.tight_layout()
    plt.show()
    print(f"Average Total Valid Points: {avg:.2f}")

team_totals_auto = team_total_valid_points(draft_with_valid, fully_autodraft_only=True)
plot_total_valid_points_dist(team_totals_auto, bins=20, title="Autodrafted Teams: Total Valid Points Distribution")

team_totals_non = team_total_valid_points(draft_with_valid, fully_autodraft_only=False)
plot_total_valid_points_dist(team_totals_non, bins=20, title="NOT 100% Autodrafted Teams: Total Valid Points Distribution")



In [ ]:
# ============================================================
# BLOCK 12 — EXPECTED VALID POINTS BY PICK (autodraft baseline)
# ============================================================

def expected_valid_points_by_pick_autodraft_distribution(draft_with_valid: pd.DataFrame) -> pd.DataFrame:
    required = ["League_ID","Year","Team","Overall","Is_Autodrafted","Season_Total_Points_Valid","Player_norm"]
    missing = [c for c in required if c not in draft_with_valid.columns]
    if missing:
        raise ValueError(f"Missing columns: {missing}")

    df = draft_with_valid.copy()

    team_auto = (
        df.groupby(["League_ID","Year","Team"], dropna=False)["Is_Autodrafted"]
          .agg(Total_Picks="size", Auto_Picks="sum")
          .reset_index()
    )
    team_auto["Is_Fully_Autodrafted"] = team_auto["Total_Picks"].eq(team_auto["Auto_Picks"])

    df = df.merge(team_auto[["League_ID","Year","Team","Is_Fully_Autodrafted"]],
                  on=["League_ID","Year","Team"], how="left")
    df = df[df["Is_Fully_Autodrafted"]].copy()

    dist = (
        df.groupby("Overall", dropna=False)
        .agg(
            Mean=("Season_Total_Points_Valid", "mean"),
            Std=("Season_Total_Points_Valid", "std"),
            N_Picks=("Season_Total_Points_Valid", "count"),
            Unique_Players=("Player_norm", "nunique"),
            P10=("Season_Total_Points_Valid", lambda s: s.quantile(0.10)),
            P25=("Season_Total_Points_Valid", lambda s: s.quantile(0.25)),
            P50=("Season_Total_Points_Valid", lambda s: s.quantile(0.50)),
            P75=("Season_Total_Points_Valid", lambda s: s.quantile(0.75)),
            P90=("Season_Total_Points_Valid", lambda s: s.quantile(0.90)),
        )
        .reset_index()
        .sort_values("Overall")
    )
    return dist

In [ ]:
# ============================================================
# BLOCK 13 — DENOISED EXPECTED CURVE (optional)
# ============================================================

def expected_valid_points_by_pick_from_autodraft_pooled_denoised(
    draft_with_valid: pd.DataFrame,
    *,
    estimator: str = "trimmed_mean",   # "mean" | "median" | "trimmed_mean" | "winsor_mean"
    trim: float = 0.10,
    smooth_window: int = 5
) -> pd.DataFrame:
    required = ["League_ID","Year","Team","Overall","Is_Autodrafted","Season_Total_Points_Valid"]
    missing = [c for c in required if c not in draft_with_valid.columns]
    if missing:
        raise ValueError(f"draft_with_valid missing columns: {missing}")

    df = draft_with_valid.copy()
    df["Overall"] = pd.to_numeric(df["Overall"], errors="coerce").astype(int)
    df["Is_Autodrafted"] = pd.to_numeric(df["Is_Autodrafted"], errors="coerce").fillna(0).astype(int)
    df["Season_Total_Points_Valid"] = pd.to_numeric(df["Season_Total_Points_Valid"], errors="coerce")

    team_auto = (
        df.groupby(["League_ID","Year","Team"], dropna=False)["Is_Autodrafted"]
          .agg(Total_Picks="size", Autodrafted_Picks="sum")
          .reset_index()
    )
    team_auto["Is_Fully_Autodrafted"] = team_auto["Total_Picks"].eq(team_auto["Autodrafted_Picks"])
    df = df.merge(team_auto[["League_ID","Year","Team","Is_Fully_Autodrafted"]],
                  on=["League_ID","Year","Team"], how="left")

    df_auto = df[df["Is_Fully_Autodrafted"]].copy()
    if df_auto.empty:
        raise ValueError("No fully-autodrafted teams found.")

    if estimator == "mean":
        agg_func = "mean"
    elif estimator == "median":
        agg_func = lambda s: float(np.nanmedian(s.to_numpy()))
    elif estimator == "trimmed_mean":
        from scipy.stats import trim_mean
        agg_func = lambda s: float(trim_mean(s.dropna().to_numpy(), proportiontocut=trim))
    elif estimator == "winsor_mean":
        from scipy.stats.mstats import winsorize
        agg_func = lambda s: float(np.mean(winsorize(s.dropna().to_numpy(), limits=trim)))
    else:
        raise ValueError("estimator must be one of: mean, median, trimmed_mean, winsor_mean")

    pooled = (
        df_auto.groupby("Overall", dropna=False)["Season_Total_Points_Valid"]
              .agg(Expected_Valid_Points=agg_func, Std_Valid_Points="std", N="count")
              .reset_index()
              .sort_values("Overall")
    )

    if smooth_window and smooth_window > 1:
        pooled["Expected_Smoothed"] = (
            pooled["Expected_Valid_Points"].rolling(window=smooth_window, center=True, min_periods=1).mean()
        )
    else:
        pooled["Expected_Smoothed"] = pooled["Expected_Valid_Points"]

    return pooled

In [ ]:
# ============================================================
# BLOCK 14 — SCORE PICKS (points added, z-score)
# ============================================================

def expected_valid_points_by_pick_from_autodraft(
    draft_with_valid: pd.DataFrame,
    *,
    by_year: bool = True
) -> pd.DataFrame:
    required = ["League_ID","Year","Team","Overall","Is_Autodrafted","Season_Total_Points_Valid"]
    missing = [c for c in required if c not in draft_with_valid.columns]
    if missing:
        raise ValueError(f"draft_with_valid missing columns: {missing}")

    df = draft_with_valid.copy()
    df["Year"] = df["Year"].astype(int)
    df["Overall"] = pd.to_numeric(df["Overall"], errors="coerce").astype(int)
    df["Is_Autodrafted"] = pd.to_numeric(df["Is_Autodrafted"], errors="coerce").fillna(0).astype(int)
    df["Season_Total_Points_Valid"] = pd.to_numeric(df["Season_Total_Points_Valid"], errors="coerce")

    team_auto = (
        df.groupby(["League_ID","Year","Team"], dropna=False)["Is_Autodrafted"]
          .agg(Total_Picks="size", Autodrafted_Picks="sum")
          .reset_index()
    )
    team_auto["Is_Fully_Autodrafted"] = team_auto["Total_Picks"].eq(team_auto["Autodrafted_Picks"])
    df = df.merge(team_auto[["League_ID","Year","Team","Is_Fully_Autodrafted"]],
                  on=["League_ID","Year","Team"], how="left")

    df_auto = df[df["Is_Fully_Autodrafted"]].copy()
    if df_auto.empty:
        raise ValueError("No fully-autodrafted teams found.")

    keys = ["Overall"] if not by_year else ["Year","Overall"]
    expected = (
        df_auto.groupby(keys, dropna=False)["Season_Total_Points_Valid"]
              .agg(Expected_Valid_Points="mean", Std_Valid_Points="std", N="count")
              .reset_index()
              .sort_values(keys)
    )
    return expected


expected_by_pick_year = expected_valid_points_by_pick_from_autodraft(draft_with_valid, by_year=True)

draft_scored = draft_with_valid.merge(
    expected_by_pick_year[["Year","Overall","Expected_Valid_Points","Std_Valid_Points"]],
    on=["Year","Overall"],
    how="left"
)
draft_scored["Points_Added"] = draft_scored["Season_Total_Points_Valid"] - draft_scored["Expected_Valid_Points"]
draft_scored["Points_Added_Z"] = draft_scored["Points_Added"] / draft_scored["Std_Valid_Points"]



In [ ]:
# ============================================================
# BLOCK 15 — PLOTS (variance band, zoom, denoised curve)
# ============================================================

# 15A: Denoised pooled curve
expected_by_pick_pooled = expected_valid_points_by_pick_from_autodraft_pooled_denoised(
    draft_with_valid,
    estimator="trimmed_mean",
    trim=0.10,
    smooth_window=5
)
expected_by_pick_pooled.plot(x="Overall", y="Expected_Smoothed",
                             title="Autodraft Expected Valid Points by Pick (All Years, Denoised)")
plt.show()

# 15B: Distribution bands (P10–P90, P25–P75)
dist = expected_valid_points_by_pick_autodraft_distribution(draft_with_valid)

plt.plot(dist["Overall"], dist["Mean"], label="Expected (Mean)")
plt.fill_between(dist["Overall"], dist["P25"], dist["P75"], alpha=0.25, label="25–75% range")
plt.fill_between(dist["Overall"], dist["P10"], dist["P90"], alpha=0.15, label="10–90% range")
plt.xlabel("Overall Pick")
plt.ylabel("Valid Season Points")
plt.title("Autodraft Expected Valid Points by Pick (with Variance)")
plt.legend()
plt.tight_layout()
plt.show()

# 15C: Zoom first 20 picks, annotate N (unique players + total picks)
dist_20 = dist[dist["Overall"] <= 20].copy()
fig, ax = plt.subplots(figsize=(16, 8))
ax.plot(dist_20["Overall"], dist_20["Mean"], label="Expected (Mean)", linewidth=2)
ax.fill_between(dist_20["Overall"], dist_20["P25"], dist_20["P75"], alpha=0.30, label="25–75% range")
ax.fill_between(dist_20["Overall"], dist_20["P10"], dist_20["P90"], alpha=0.18, label="10–90% range")
ax.set_xlabel("Overall Pick", fontsize=12)
ax.set_ylabel("Valid Season Points", fontsize=12)
ax.set_title("Autodraft Expected Valid Points by Pick\nFirst 20 Picks (Variance Zoom)", fontsize=14)

for _, r in dist_20.iterrows():
    ax.text(
        r["Overall"], r["P10"] - 5,
        f"{int(r['Unique_Players'])} players\n(n={int(r['N_Picks'])})",
        ha="center", va="top", fontsize=9, alpha=0.7
    )

ax.legend()
ax.grid(alpha=0.2)
fig.tight_layout()
plt.show()

print(expected_by_pick_pooled[["Overall","Expected_Valid_Points","Expected_Smoothed","N"]].head(10))
print(expected_by_pick_pooled[["Overall","Expected_Valid_Points","Expected_Smoothed","N"]].tail(10))